In [ ]:
!pip install influxdb-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.3/746.3 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.6/218.6 kB 15.0 MB/s eta 0:00:00


In [ ]:
bucket = "crop_sensor_data"
org = "MyAI/ML_Workspace"
token = "Vyp0JkjzoP7cwkE491wtCOKG_KVyai5-cfn8C1Y2_IhGw0oWiXm58_Icln0iWHqwzbzfskov_3zwIv7B4cf3xQ=="
url = "https://us-east-1-1.aws.cloud2.influxdata.com"

In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 6.5 MB/s eta 0:00:00


In [ ]:
from influxdb_client import InfluxDBClient
from groq import Groq
import time

# -------------------------------
# InfluxDB Configuration
# -------------------------------
bucket = "crop_sensor_data"
org = "MyAI/ML_Workspace"
token = "xx"
url = "https://us-east-1-1.aws.cloud2.influxdata.com"

# -------------------------------
# Groq Configuration
# -------------------------------
GROQ_API_KEY = "xx"

# -------------------------------
# Initialize Clients
# -------------------------------
influx_client = InfluxDBClient(url=url, token=token, org=org)
query_api = influx_client.query_api()

groq_client = Groq(api_key=GROQ_API_KEY)

# -------------------------------
# Fetch Latest Sensor Data
# -------------------------------
def fetch_latest_data():
    query = f'''
    from(bucket:"{bucket}")
      |> range(start: -30d)
      |> filter(fn: (r) => r._measurement == "crop_sensors")
      |> last()
    '''

    tables = query_api.query(query)

    records = {}
    for table in tables:
        for record in table.records:
            records[record.get_field()] = record.get_value()

    return records



# -------------------------------
# Build LLM Prompt
# -------------------------------
def build_prompt(sensor_data):
    prompt = f"""
You are an agriculture AI assistant.

Analyze the crop sensor data below and respond in ONLY 3–4 short lines.

Sensor Data:
Temperature: {sensor_data.get("temperature")} °C
Humidity: {sensor_data.get("humidity")} %
Soil Moisture: {sensor_data.get("soil_moisture")} %
Light Intensity: {sensor_data.get("light_intensity")} lux
Soil pH: {sensor_data.get("ph")}
Nitrogen Level: {sensor_data.get("nitrogen")} ppm

Format your response exactly as:
1. Crop Health:
2. Risk:
3. Irrigation:
4. Fertilizer:
"""
    return prompt



# -------------------------------
# Ask Groq LLM
# -------------------------------
def ask_llm(prompt):
    response = groq_client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "You are an agriculture AI expert."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content


# -------------------------------
# Main Loop (Real-Time AI Engine)
# -------------------------------
def main():
    print("🌾 AI Crop Monitoring System Started...\n")

    while True:
        try:
            sensor_data = fetch_latest_data()

            if not sensor_data:
                print("Waiting for sensor data...")
                time.sleep(10)
                continue

            prompt = build_prompt(sensor_data)
            ai_response = ask_llm(prompt)

            print("\n================ AI CROP ANALYSIS ================")
            print(ai_response)
            print("=================================================\n")

            time.sleep(30)   # Run AI analysis every 30 seconds

        except Exception as e:
            print("Error:", e)
            time.sleep(10)


# -------------------------------
# Run Program
# -------------------------------
if __name__ == "__main__":
    main()


🌾 AI Crop Monitoring System Started...


================ AI CROP ANALYSIS ================
1. Crop Health: Mild stress symptoms due to high temperature and moderate soil moisture.
2. Risk: Potential nitrogen deficiency, moderate pest or disease risk.
3. Irrigation: Suggest watering 2-3 times this week to maintain optimal soil moisture.
4. Fertilizer: Apply nitrogen-rich fertilizer to raise nitrogen levels by 10-20 ppm.


================ AI CROP ANALYSIS ================
1. Crop Health: Fair
2. Risk: Water stress risk, high soil pH
3. Irrigation: Immediate watering required
4. Fertilizer: Apply nitrogen at 20 ppm to avoid overfertilization


================ AI CROP ANALYSIS ================
1. Crop Health: Generally healthy but slightly stressed due to heat.
2. Risk: Risk of heat stress and nitrogen deficiency.
3. Irrigation: Soil moisture at adequate level, but irrigation may be required soon.
4. Fertilizer: Nitrogen levels could be increased to promote healthy growth.


===========